In [1]:
# importing packages

from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS, LabelSet, Label
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Panel, Tabs, Button
from bokeh.models.glyphs import Text
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import matplotlib.pyplot as mplt
import matplotlib.lines as mlines
import time

output_notebook()

import numpy as np
import pandas as pd
from itertools import permutations
from gurobipy import *

Loading BokehJS ...

In [2]:
# settings

R = 9 # number of matrices max:8
M = 8 # number of rows max:20
N = 12 # number of columns max:20
K = 9 # number of partitions

F = N # flexibility of shapes
H = M # height of shapes
W = N # width of shapes

In [3]:
xls = pd.ExcelFile('data.xlsx')
df = [None]*9
df[0] = pd.read_excel(xls, 'Water Pipe (Cold Hot)', header=None)
df[1] = pd.read_excel(xls, 'High Conduit', header=None)
df[2] = pd.read_excel(xls, 'Rectangular Duct', header=None)
df[3] = pd.read_excel(xls, 'CAV-VAV', header=None)
df[4] = pd.read_excel(xls, 'M-Pipe-Wet & Connections', header=None)
df[5] = pd.read_excel(xls, 'Round Duct', header=None)
df[6] = pd.read_excel(xls, 'Cable Tray', header=None)
df[7] = pd.read_excel(xls, 'Conduit Runs', header=None)
df[8] = pd.read_excel(xls, 'Electrical CAV-VAV', header=None)

In [4]:
b = [[]] * R
for r in range(R):
    b[r] = np.array(df[r])

In [5]:
# parameters

np.random.seed(2)

a = [[]] * R

for r in range(R):
    a[r] = np.zeros((M, N+1))
    
for r in range(R):
    for i in range(M):
        for j in range(1, N+1):
            a[r][i][j] = a[r][i][j-1] + b[r][i][j-1]
            
Si = np.array(range(0,M,1))
Sj = np.array(range(0,N+1,1))
Sk = np.array(range(0,K+1,1))
Sr = np.array(range(0,R,1))

In [6]:
TimeTotal = time.time()

# model
BCP = Model('BCP_q model')

# variables
x = BCP.addVars(Si, Sj, Sk, lb=0.0, ub=1.0, vtype='B', name="X")
ux = BCP.addVars(Si, Sj, Sk, lb=0.0, vtype='C', name="X")
y = BCP.addVars(Si, Sk, lb=0.0, ub=1.0, vtype='B', name="Y")
B = BCP.addVars(Sk, lb=0.0, ub=1.0, vtype='B', name="B")
BB = BCP.addVars(Sk, lb=0.0, ub=1.0, vtype='B', name="B")
s = BCP.addVars(Si, Sk, lb=0.0, ub=1.0, vtype='B', name="S")
z = BCP.addVars(Sr, Sk, lb=0.0, vtype='C', name="Z")
OBJ = BCP.addVar(lb=0.0, vtype='C', name="OBJECTIVE")

# constraint 0
BCP.addConstrs(x[i,0,0] == 1 for i in Si)
BCP.addConstrs(x[i,N,K] == 1 for i in Si)
BCP.update()

# constraint 1
BCP.addConstrs((
    quicksum(x[i, j, k] for j in Sj) == 1 
    for i in Si
    for k in Sk),
    name = "exactly k cuts per row")
BCP.update()

# constraint 2c
BCP.addConstrs((
    ux[i,j,k] >= x[i,j,k] + y[i,k] - 1
    for n in Sj
    for i in Si
    for k in Sk),
    name = "ux")
BCP.update()

# constraint 2''
BCP.addConstrs((
    quicksum(x[i,j,k-1] - x[i,j,k] for j in Sj[:n])>= ux[i,n,k]
    for n in Sj[1:]
    for i in Si
    for k in Sk[1:]),
    name = "Order of cuts SSS")
BCP.update()

# # constraint 3
BCP.addConstrs((
    W * y[i, k] >= quicksum(j*(x[i,j,k]-x[i,j,k-1]) for j in Sj) 
    for i in Si
    for k in Sk[1:]),
    name = "Openness_1")
BCP.addConstrs((
    y[i, k] <= quicksum(j*(x[i,j,k]-x[i,j,k-1]) for j in Sj) 
    for i in Si
    for k in Sk[1:]),
    name = "Openness_2")
BCP.update()

# # constraint 3''
BCP.addConstrs((
    quicksum(x[i,j,k-1] for j in Sj[:n]) <= quicksum(x[i,j,k] for j in Sj[:n]) + y[i, k]
    for n in Sj[1:]
    for i in Si
    for k in Sk[1:]),
    name = "Openness_1 SSS")
BCP.update()

# constraint 4
BCP.addConstrs((
    s[i,k] + y[i-1,k] >= y[i,k] 
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Starts")
BCP.addConstrs((
    s[0,k] >= y[0,k]
    for k in Sk[1:]),
    name = "Starts_0")
BCP.update()

# constraint 5
BCP.addConstrs((
    quicksum(s[i,k] for i in Si) == 1
    for k in Sk[1:]),
    name = "One start for each partition")
BCP.update()

# constraint 5'
BCP.addConstrs((
    quicksum(s[i,k]*(nn+1) - y[i,k] for i in Si[:nn+1]) >= 0
    for nn in Si
    for k in Sk[1:]),
    name = "Maybe a cut")
BCP.update()

# constraint 6
BCP.addConstrs((
    quicksum(j*(x[i-1,j,k-1]) for j in Sj) <= quicksum(j*(x[i,j,k]) for j in Sj) - 1 + M * (2-y[i-1,k]-y[i,k])
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_1")
BCP.addConstrs((
    quicksum(j*(x[i,j,k-1]) for j in Sj) <= quicksum(j*(x[i-1,j,k]) for j in Sj) - 1 + M * (2-y[i-1,k]-y[i,k])
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_2")
BCP.update()

# constraint 6''a
BCP.addConstrs((
    quicksum(x[i-1,j,k-1] for j in Sj[:n]) >= quicksum(x[i,j,k] for j in Sj[:n+1]) + y[i-1,k] + y[i,k] - 2
    for n in Sj[1:-1]
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_1 SSS")

# constraint 6''b
BCP.addConstrs((
    quicksum(x[i,j,k-1] for j in Sj[:n]) >= quicksum(x[i-1,j,k] for j in Sj[:n+1]) + y[i-1,k] + y[i,k] - 2
    for n in Sj[1:-1]
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_2 SSS")
BCP.update()




#-----------------------------------



BCP.addConstrs((
    quicksum(x[i-1,jj,k] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i,j,k] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_1 FLEX")


BCP.addConstrs((
    quicksum(x[i,jj,k] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i-1,j,k] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_2 FLEX")



BCP.addConstrs((
    quicksum(x[i-1,jj,k-1] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i,j,k-1] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_3 FLEX")


BCP.addConstrs((
    quicksum(x[i,jj,k-1] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i-1,j,k-1] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_4 FLEX")



#-----------------------------------



# constraint QuasiConvex
BCP.addConstrs((
    quicksum(x[i-1,jj,k] for jj in Sj[max(j-F,0):j+1]) >= x[i,j,k] + y[i-1,k] + y[i,k] - 2 - B[k]
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_1 QuasiN")
BCP.addConstrs((
    quicksum(x[i-1,jj,k] for jj in Sj[j:min(j+F+1,N+1)]) >= x[i,j,k] + y[i-1,k] + y[i,k] - 2 + B[k] - 1
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_1 QuasiP")

BCP.addConstrs((
    quicksum(x[i,jj,k] for jj in Sj[j:min(j+F+1,N+1)]) >= x[i-1,j,k] + y[i-1,k] + y[i,k] - 2 - B[k]
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_2 QuasiN")
BCP.addConstrs((
    quicksum(x[i,jj,k] for jj in Sj[max(j-F,0):j+1]) >= x[i-1,j,k] + y[i-1,k] + y[i,k] - 2 + B[k] - 1
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_2 QuasiP")


BCP.addConstrs((
    quicksum(x[i-1,jj,k-1] for jj in Sj[max(j-F,0):j+1]) >= x[i,j,k-1] + y[i-1,k] + y[i,k] - 2 + B[k] - 1
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_3 QuasiN")
BCP.addConstrs((
    quicksum(x[i-1,jj,k-1] for jj in Sj[j:min(j+F+1,N+1)]) >= x[i,j,k-1] + y[i-1,k] + y[i,k] - 2 - B[k] 
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_3 QuasiP")

BCP.addConstrs((
    quicksum(x[i,jj,k-1] for jj in Sj[j:min(j+F+1,N+1)]) >= x[i-1,j,k-1] + y[i-1,k] + y[i,k] - 2 + B[k] - 1
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_4 QuasiN")
BCP.addConstrs((
    quicksum(x[i,jj,k-1] for jj in Sj[max(j-F,0):j+1]) >= x[i-1,j,k-1] + y[i-1,k] + y[i,k] - 2 - B[k]
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_4 QuasiP")


#-----------------------------------




BCP.addConstrs((
    quicksum(y[i,k] for i in Si) <= H
    for k in Sk),
    name = "Height")


# constraint 7
BCP.addConstrs((
    z[r,k] == quicksum(a[r][i][j]*(x[i,j,k]-x[i,j,k-1]) for j in Sj for i in Si)
    for r in Sr
    for k in Sk[1:]),
    name = "Sum of the partition")

# constraint 7''
BCP.addConstrs((
    OBJ >= z[r,k]
    for r in Sr
    for k in Sk[1:]),
    name = "OBJ on Sum of the partition")

BCP.update()

# objective
BCP.setObjective(OBJ, GRB.MINIMIZE)

# solution
BCP.optimize()

ttime = time.time()-TimeTotal
print('\n\n\nTotal - Time = {}     Obj = {}'.format(round(ttime, 1), round(BCP.ObjVal, 2)))

Academic license - for non-commercial use only
Optimize a model with 15219 rows, 2351 columns and 198480 nonzeros
Variable types: 1131 continuous, 1220 integer (1220 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 6862 rows and 1238 columns
Presolve time: 0.39s
Presolved: 8357 rows, 1113 columns, 99509 nonzeros
Variable types: 79 continuous, 1034 integer (1025 binary)
Found heuristic solution: objective 13.2937000

Root relaxation: objective 1.989767e+00, 1635 iterations, 0.20 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.98977    0  134   13.29370    1.98977  85.0%     -    1s
H    0     0                       4.4000000    1.98977  54.8%     -    1s
H    0     0                       4.3269000    1.98977  54.0%     -   

H95233 17924                       2.0580000    1.98977  3.32%   176  490s
 95256 17464    1.99546   53  116    2.05800    1.98977  3.32%   176  502s
 99014 18417    1.98977   52  133    2.05800    1.98977  3.32%   177  512s
H100213 18804                       2.0579996    1.98977  3.32%   177  512s
 102247 19147    2.02219   68  118    2.05800    1.98977  3.32%   178  522s
 103893 19128 infeasible   54         2.05800    1.98977  3.32%   178  535s
 107254 19727 infeasible   67         2.05800    1.98977  3.32%   179  545s
 110378 20342     cutoff   69         2.05800    1.98977  3.32%   179  555s
 112123 20342 infeasible   80         2.05800    1.98977  3.32%   179  565s
 115048 20950    2.02640   64   86    2.05800    1.98977  3.32%   180  574s
 115164 20987    1.99345   65  184    2.05800    1.98977  3.32%   180  575s
H115847 21246                       2.0579996    1.98977  3.32%   180  575s
 117987 21840 infeasible   75         2.05800    1.98977  3.32%   181  586s
 118984 21805  

 284995 30813    1.98977   76  125    2.04410    1.98977  2.66%   207 1392s
 287285 30450     cutoff   83         2.04410    1.98977  2.66%   207 1400s
 289677 30136     cutoff   93         2.04410    1.98977  2.66%   207 1411s
 291966 29959    1.99376   78   77    2.04410    1.98977  2.66%   208 1424s
 292872 29529 infeasible   67         2.04410    1.98977  2.66%   208 1434s
 295844 28848 infeasible   72         2.04410    1.98977  2.66%   208 1445s
 298455 28070    1.98977   71   97    2.04410    1.98977  2.66%   209 1455s
 300872 27539 infeasible   77         2.04410    1.98977  2.66%   209 1466s
 301983 26832    1.99763   98  151    2.04410    1.98977  2.66%   209 1478s
 304531 26117     cutoff   68         2.04410    1.98977  2.66%   209 1489s
 307267 25360    1.98977   70   97    2.04410    1.98977  2.66%   210 1500s
 310017 24972 infeasible   83         2.04410    1.98977  2.66%   210 1513s
H310066 22351                       2.0349000    1.98977  2.22%   210 1513s
 310940 2178

In [7]:
# drawing Matrix

c_m = np.zeros(shape=(M,K+1)).astype(int)


for i in range(M):
    for j in range(N+1):
        for k in range(K+1):
            c_m[i,k] += round(x[i,j,k].x*j)


col = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
           '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
           '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']


plt = figure(plot_width=500, plot_height=round(500*M/N))


for i in range(0,M):
    for k in range(1,K+1):
        plt.quad(top=[-i], bottom=[-i-1], left=[c_m[i,k-1]], right=[c_m[i,k]], color=col[(k-1)%20], alpha = 0.4)



# for i in range(-m,0):
#     for j in range(1,n+1):
#         plt.quad(top=[i+0.5], bottom=[i-00.5], left=[j-0.5], right=[j+0.5], color=col[a_m[-i-1,j-1]%20])
   
    
for i in range(0,M+1):
    plt.line([0, N], [-i, -i], line_color='black', line_width=1, alpha = 1)
for i in range(0,N+1):
    plt.line([i, i], [0, -M], line_color='black', line_width=1, alpha = 1)


print(BCP.ObjVal)
pltName = "TEST.html"
output_file(pltName)
show(plt)

2.0348998627102675


In [8]:
for v in BCP.getVars():
    if (v.x != 0):
        print(v.VarName, round(v.x,2))

X[0,0,0] 1.0
X[0,0,1] 1.0
X[0,0,2] 1.0
X[0,0,3] 1.0
X[0,0,4] 1.0
X[0,0,5] 1.0
X[0,0,6] 1.0
X[0,0,7] 1.0
X[0,9,8] 1.0
X[0,12,9] 1.0
X[1,0,0] 1.0
X[1,5,1] 1.0
X[1,5,2] 1.0
X[1,5,3] 1.0
X[1,5,4] 1.0
X[1,5,5] 1.0
X[1,5,6] 1.0
X[1,5,7] 1.0
X[1,7,8] 1.0
X[1,12,9] 1.0
X[2,0,0] 1.0
X[2,3,1] 1.0
X[2,3,2] 1.0
X[2,7,3] 1.0
X[2,7,4] 1.0
X[2,7,5] 1.0
X[2,7,6] 1.0
X[2,12,7] 1.0
X[2,12,8] 1.0
X[2,12,9] 1.0
X[3,0,0] 1.0
X[3,3,1] 1.0
X[3,3,2] 1.0
X[3,6,3] 1.0
X[3,6,4] 1.0
X[3,6,5] 1.0
X[3,11,6] 1.0
X[3,12,7] 1.0
X[3,12,8] 1.0
X[3,12,9] 1.0
X[4,0,0] 1.0
X[4,2,1] 1.0
X[4,4,2] 1.0
X[4,5,3] 1.0
X[4,10,4] 1.0
X[4,10,5] 1.0
X[4,11,6] 1.0
X[4,12,7] 1.0
X[4,12,8] 1.0
X[4,12,9] 1.0
X[5,0,0] 1.0
X[5,1,1] 1.0
X[5,5,2] 1.0
X[5,5,3] 1.0
X[5,7,4] 1.0
X[5,10,5] 1.0
X[5,11,6] 1.0
X[5,12,7] 1.0
X[5,12,8] 1.0
X[5,12,9] 1.0
X[6,0,0] 1.0
X[6,0,1] 1.0
X[6,7,2] 1.0
X[6,7,3] 1.0
X[6,7,4] 1.0
X[6,11,5] 1.0
X[6,11,6] 1.0
X[6,12,7] 1.0
X[6,12,8] 1.0
X[6,12,9] 1.0
X[7,0,0] 1.0
X[7,0,1] 1.0
X[7,0,2] 1.0
X[7,0,3] 1.0
X[7,0,4] 1.0


In [9]:
R_name = ['Water Pipe (Cold Hot)', 'High Conduit', 'Rectangular Duct', 'CAV-VAV', 'M-Pipe-Wet & Connections',\
          'RoundDuct', 'CableTray', 'Conduit Runs', 'Electrical CAV-VAV']

col_rct = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
       '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
       '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

col = ['#000075','#42d4f4','#f58231','#8A2BE2','#A52A2A','#f032e6','#4363d8','#808000', 'green']



partitions = []
for i in range(1,K+1):
    partitions.append("Partition_" + str(i))

data = {}
data['prtitions'] = partitions
for r in range(R):
    data[R_name[r]] = []
    for k in range(K):
        data[R_name[r]].append(z[r,k+1].x)

w = 2.8/(5*R-1)
source = ColumnDataSource(data=data)

p = figure(x_range=partitions, y_range=(0, BCP.ObjVal*1.5), plot_height=350, plot_width=1250,\
           title="TAKT planning result", tools="save")

for k in range(K):
    p.quad(top=[BCP.ObjVal*1.3], bottom=[0], left=[k], right=[k+1], color=col_rct[k], alpha = 0.2)    

for r in range(R):
    p.vbar(x=dodge('prtitions', w*5/4*r-0.35+w/2, range=p.x_range), top=R_name[r], width=w, source=source,
       color=col[r], legend=value(R_name[r]))

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)